In [9]:
import pandas as pd
 # Load merged and cleaned dataset
merged = pd.read_csv("merged_clean.csv")
rename_map = {
    "product_name_product": "product_name",
    "brand_name_product": "brand_name",
    "price_usd_product": "price_usd",
}
existing = {k: v for k, v in rename_map.items() if k in merged.columns}
if existing:
    merged = merged.rename(columns=existing)

merged = merged[merged["secondary_category"] != "Shop by Concern"]
merged["rating_review"] = pd.to_numeric(merged["rating_review"], errors="coerce")
min_reviews = 200

product_stats = (
    merged
    .groupby(["secondary_category", "product_id", "product_name"])
    .agg(
        avg_rating=("rating_review", "mean"),
        review_count=("rating_review", "count")
    )
    .reset_index()
)

product_stats = product_stats[product_stats["review_count"] >= min_reviews]
product_stats["avg_rating"] = product_stats["avg_rating"].round(2)

/var/folders/tf/mcm346y959j6c4rtwh8wm3280000gn/T/ipykernel_77100/3974004709.py:3: DtypeWarning: Columns (0,1,2,4,5,6,8) have mixed types. Specify dtype option on import or set low_memory=False.
  merged = pd.read_csv("merged_clean.csv")


In [10]:
# Normalization and Best Seller Score Calculation
product_stats["rating_norm"] = product_stats["avg_rating"] / 5

product_stats["reviews_norm"] = (
    product_stats
    .groupby("secondary_category")["review_count"]
    .transform(lambda x: x / max(1, x.max()))
)

product_stats["best_seller_score"] = (
    0.6 * product_stats["reviews_norm"] +
    0.4 * product_stats["rating_norm"]
)

product_stats["best_seller_score"] = product_stats["best_seller_score"].round(3)

In [11]:
product_stats["rank_in_secondary_category"] = (
    product_stats
    .groupby("secondary_category")["best_seller_score"]
    .rank(method="dense", ascending=False)
)
top_products_per_category = (
    product_stats[product_stats["rank_in_secondary_category"] <= 5]
    .sort_values(["secondary_category", "rank_in_secondary_category"])
)

top_products_per_category

top_products_per_category.to_csv(
    "outputs/best_sellers_per_secondary_category.csv",
    index=False
)


In [12]:
final_best_sellers = (
    top_products_per_category
    .copy()
)

final_best_sellers["avg_rating"] = final_best_sellers["avg_rating"].round(2)
final_best_sellers["rank_in_secondary_category"] = final_best_sellers["rank_in_secondary_category"].astype(int)

final_best_sellers = final_best_sellers[
    ["secondary_category", "rank_in_secondary_category", "product_id", "product_name", "avg_rating", "review_count"]
].sort_values(["secondary_category", "rank_in_secondary_category"])

final_best_sellers.to_csv("outputs/best_sellers_clean.csv", index=False)
final_best_sellers.head(20)


,secondary_category,rank_in_secondary_category,product_id,product_name,avg_rating,review_count
34,Cleansers,1,P386197,GinZing Refreshing Scrub Cleanser,4.98,37399
57,Cleansers,2,P409631,SEA FRXXXTION Stick Exfoliating Cleanser,4.99,33217
23,Cleansers,3,P297516,Checks and Balances Frothy Face Wash,4.93,31661
40,Cleansers,4,P392248,Pekee Cleansing Bar,4.99,28091
349,Cleansers,5,P7880,Soy Hydrating Gentle Face Cleanser,4.36,8736
379,Eye Care,1,P406104,24K Gold Pure Luxury Lift & Firm Hydra-Gel Eye...,4.99,62980
446,Eye Care,2,P459129,Peptide Eye Cream,4.82,268
489,Eye Care,2,P476414,5 Stars Retinol + Niacinamide Eye Serum,4.64,1721
535,Eye Care,3,P505049,barrier+ Triple Lipid + Collagen Brightening E...,4.80,277
395,Eye Care,4,P422905,Moisturizing Eye Bomb with Squalane,4.46,2966
